In [19]:

import logging
import re
import nltk
import numpy as np


#Tokenizer
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('stopwords')

logger = logging.getLogger('FraudEmails')

frequency = 100

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\jseme\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\jseme\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\jseme\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\stopwords.zip.


In [2]:
#Open and convert file to String
try:
    with open('../data/fraudulent_emails.txt','r') as file:
        text = file.read()
except Exception as e:
    logger.error('Process failed with error: '+repr(e))
finally:
    file.close()

In [14]:
#Delete everything between From r [.*?] Status: ?O
formatEmails = re.sub('From.*?Status: ?O','',text,flags=re.DOTALL)


In [15]:
#Convert to lower case
formatEmails = formatEmails.lower()

In [20]:
tokenizer = nltk.tokenize.RegexpTokenizer(r'\w+')
tokens = tokenizer.tokenize(formatEmails)
words = filter(lambda tok: tok not in nltk.corpus.stopwords.words('english'), tokens)

processed_emails = ' '.join(words)

In [22]:
characters = sorted(list(set(processed_emails)))
numb_chars = dict((c,i) for i,c in enumerate(characters))

In [24]:
numb_words = len(processed_emails)
print('Number of characters:',numb_words)

Character length: 3052533


In [26]:
numb_vocab = len(numb_chars)
print('Number of different types of characters:',numb_vocab)

Number of different types of characters: 53


In [27]:
#Length of an input sequence mapping
sequence_length = 100
x = []
y = []

In [28]:

for i in range(numb_words-sequence_length):
    #Current character to sequence length
    input = processed_emails[i:i+sequence_length]
    #Initial character to final sequence length
    output = processed_emails[i+sequence_length]

    #Chars to ints
    x.append([numb_chars[char] for char in input])
    y.append(numb_chars[output])

In [29]:
patterns = len(x)
print('Number of patterns:',patterns)

Number of patterns: 3052433
